This notebook presents the quantization process of the [meta-llama/Llama-2-7b-chat-hf](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) model using [AutoAWQ](https://github.com/casper-hansen/AutoAWQ). The resulting quantized model can be accessed at [CalvinU/Llama-2-7b-chat-hf-awq](https://huggingface.co/CalvinU/Llama-2-7b-chat-hf-awq).

In [ ]:
%pip install -q transformers autoawq accelerate

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = "meta-llama/Llama-2-7b-chat-hf"
quant_path = "Llama-2-7b-chat-hf-awq"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}

# Load model and tokenizer
model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage": True})
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize model
model.quantize(tokenizer, quant_config=quant_config)

In [ ]:
from transformers import AwqConfig

# Create a Transformers-compatible quantization configuration
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# Pass the new quantization configuration to the quantized model
model.model.config.quantization_config = quantization_config

# Save the quantized model weights
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path=quant_path,
    repo_id="CalvinU/Llama-2-7b-chat-hf-awq",
    repo_type="model",
)